In [1]:
# import module
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import re
path = "D:/driver/chromedriver.exe"    #ex. C:/downloads/chromedriver.exe

## 측정 지역명 추출

크롤링 위해 기존에 제공하는 데이터 셋에서 인천 23개 지역 측정 명 추출

인천 남구는 현재 검색 불가, 미추홀구로 변경되어 해당 value replace하여 진행

주안 6동 위치도 마찬가지로 검색 안됨, 근처의 석암파출소로 변경 후 진행

In [16]:
pm_final = pd.read_csv('D:/data/pm_data_korea.csv', encoding='CP949')
pm_final['지역'] = pm_final['지역'].str[:2] 
pm_final = pm_final[pm_final['지역'] == '인천']
pm_final['주소'] =pm_final['주소'].apply(lambda x : re.split(r'\(',x)[0])
site_list = ['송해', '길상', '운서', '검단', '원당', '연희', '청라', '석남', '계산', '삼산', '부평',
            '부평역', '송현', '신흥', '동춘', '송도', '숭의', '석바위', '구월동', '고잔', '논현']
pm_final = pm_final[pm_final['측정소명'].isin(site_list)]

# 검색결과 1000건 이상 혹은 검색안되는 주소 변경
pm_final['주소'] =pm_final['주소'].str.replace('남구','미추홀구')
pm_final['주소'] =pm_final['주소'].str.replace('주안6동','석암파출소')
pm_final['주소'] =pm_final['주소'].str.replace('송현동','수문통로 77')
pm_final['주소'] =pm_final['주소'].str.replace('131','') # 크리스탈로
incheon_list =pm_final['주소'].unique().tolist()

In [17]:
pm_final['주소'].unique().tolist()

['인천 중구 서해대로 471',
 '인천 남동구 구월말로 7',
 '인천 미추홀구 독정이로 95',
 '인천 미추홀구 석암파출소',
 '인천 부평구 광장로 지하15',
 '인천 부평구 부평대로88번길 19',
 '인천 서구 심곡로 98',
 '인천 서구 검단로502번길 15',
 '인천 계양구 계양산로134번길 18',
 '인천 남동구 남동대로 217',
 '인천 서구 거북로 116',
 '인천 강화군 송해면 전망대로 29',
 '인천 연수구 원인재로 115',
 '인천 중구 영종대로 85',
 '인천 동구 수문통로 77',
 '인천 남동구 청능대로611번길 54',
 '인천 서구 고산후로121번길 7',
 '인천광역시 미추홀구 독정안길 26',
 '인천광역시 동구 수문통로 77',
 '인천 서구 크리스탈로 ',
 '인천광역시 연수구 갯벌로 12',
 '인천광역시 남동구 은봉로 82']

In [10]:
pm_final['측정소명'].unique().tolist()

['신흥',
 '구월동',
 '숭의',
 '석바위',
 '부평역',
 '부평',
 '연희',
 '검단',
 '계산',
 '고잔',
 '석남',
 '송해',
 '동춘',
 '운서',
 '송현',
 '논현',
 '원당',
 '청라',
 '송도']

In [11]:
len(incheon_list)

22

## 에어코리아 실시간 자료 조회 -> 우리동네 대기정보 데이터 다운

셀레니움을 활용하여 지역명 검색 클릭, 도로명 주소 입력, 측정자료 조회기간 설정, 엑셀 다운을 하는 코드 생성

경기도 이외의 지역은 에어코리아에서 자료를 받아야 하기 때문에 통계정보 -> 최종확정측정자료조회에 있는 과거데이터,

오픈 API를 활용한 최근 3개월 데이터 수집, 

우리동네 대기정보에 있는 위의 두 구간에 포함되지 않는 데이터 3 종류의 데이터를 수집 후 전처리 진행

In [18]:
# Selenium Load
driver = webdriver.Chrome(path)
# site connect
driver.get('https://www.airkorea.or.kr/web/realSearch?pMENU_NO=97')

for i in incheon_list:
    driver.find_element_by_xpath('//*[@id="s_condition_input3"]').click()
    driver.implicitly_wait(4)

    #chrome switch
    driver.switch_to_window(driver.window_handles[1])
    driver.get_window_position(driver.window_handles[1])
    driver.implicitly_wait(4)

    # site select
    driver.find_element_by_xpath('//*[@id="keyword"]').send_keys(i)
    driver.find_element_by_xpath('//*[@id="serarchContentBox"]/fieldset/span/input[2]').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="roadAddrDiv1"]/b').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="resultData"]/div/a').click()
    time.sleep(3)

    #chrome switch
    driver.switch_to_window(driver.window_handles[0])
    driver.get_window_position(driver.window_handles[0])

    #측정자료 조회 날짜 설정(10월1일 ~11월30일)
    driver.find_element_by_xpath('//*[@id="cont_body"]/form[2]/div[1]/a').click()
    time.sleep(4)
    driver.find_element_by_xpath('//*[@id="from_date"]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[7]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[10]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td[3]/a').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="to_date"]').click()
    time.sleep(2)
    
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[7]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[11]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[5]/td[7]/a').click()
    time.sleep(4)
    driver.find_element_by_xpath('//*[@id="to_date_hour"]/option[24]').click()
    #검색 버튼 클릭
    driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[1]').click()
    time.sleep(4)

    #Save file
    driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[3]').click()
    time.sleep(5)
    
    #측정자료 조회 날짜 설정(12월1일 ~12월31일)
    driver.find_element_by_xpath('//*[@id="from_date"]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[7]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[12]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td[1]/a').click()
    time.sleep(4)
    driver.find_element_by_xpath('//*[@id="to_date"]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[7]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[12]').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[5]/td[3]/a').click()
    time.sleep(4)
    driver.find_element_by_xpath('//*[@id="to_date_hour"]/option[24]').click()
    #검색 버튼 클릭
    driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[1]').click()
    time.sleep(4)

    #Save file
    driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[3]').click()
    time.sleep(8)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use driver.switch_to.window instead
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: use driver.switch_to.window instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="s_condition_input3"]"}
  (Session info: chrome=79.0.3945.117)


## 충남 당진

In [69]:
pm_chungnam = pd.read_csv('D:/data/pm_data_korea.csv', encoding='CP949')
pm_chungnam['주소'] =pm_chungnam['주소'].apply(lambda x : re.split(r'\(',x)[0])
pm_chungnam = pm_chungnam[pm_chungnam['측정소명']=='송산면']
chungnam_list =pm_chungnam['주소'].unique().tolist()


C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,2,3,4,5,6,7,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
chungnam_list

['충남 당진시 송산면 유곡로 342-27']

In [71]:
# Selenium Load
driver = webdriver.Chrome(path)
# site connect
driver.get('https://www.airkorea.or.kr/web/realSearch?pMENU_NO=97')

driver.find_element_by_xpath('//*[@id="s_condition_input3"]').click()
driver.implicitly_wait(10)

#chrome switch
driver.switch_to_window(driver.window_handles[1])
driver.get_window_position(driver.window_handles[1])
driver.implicitly_wait(10)

# site select
driver.find_element_by_xpath('//*[@id="keyword"]').send_keys('충남 당진시 송산면 유곡로 342-27')
driver.find_element_by_xpath('//*[@id="serarchContentBox"]/fieldset/span/input[2]').click()
time.sleep(3)
driver.find_element_by_xpath('//*[@id="roadAddrDiv1"]/b').click()
time.sleep(5)
driver.find_element_by_xpath('//*[@id="resultData"]/div/a').click()
time.sleep(3)

#chrome switch
driver.switch_to_window(driver.window_handles[0])
driver.get_window_position(driver.window_handles[0])

#측정자료 조회 날짜 설정(5월1일 ~6월30일)
driver.find_element_by_xpath('//*[@id="cont_body"]/form[2]/div[1]/a').click()
time.sleep(10)
driver.find_element_by_xpath('//*[@id="from_date"]').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[5]').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td[4]/a').click()
time.sleep(10)
driver.find_element_by_xpath('//*[@id="to_date"]').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[6]').click()
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[6]/td[1]/a').click()
time.sleep(4)
driver.find_element_by_xpath('//*[@id="to_date_hour"]/option[24]').click()
#검색 버튼 클릭
driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[1]').click()
time.sleep(4)

#Save file
driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[3]').click()
time.sleep(10)

#측정자료 조회 날짜 설정(7월1일 ~8월30일)
driver.find_element_by_xpath('//*[@id="from_date"]').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[7]').click()
time.sleep(1)
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td[2]/a').click()
time.sleep(10)
driver.find_element_by_xpath('//*[@id="to_date"]').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[8]').click()
driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[3]/td[4]/a').click()
time.sleep(4)
driver.find_element_by_xpath('//*[@id="to_date_hour"]/option[24]').click()
#검색 버튼 클릭
driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[1]').click()
time.sleep(5)

#Save file
driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[3]').click()
time.sleep(10)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use driver.switch_to.window instead
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: use driver.switch_to.window instead


## 경기도 크롤링

In [8]:
address_list = ['경기 수원시 영통구 법조로 129', '경기 수원시 권선구 서부로 1600', '경기 광명시 시청로 20', 
                '경기 안산시 단원구 화랑로 387', '경기 안산시 단원구 산단로 112 ', '경기 의왕시 시청로 11',
                '경기 시흥시 공단1대로 204', '경기 시흥시 복지로 37', '경기 군포시 청백리길 6', '경기 부천시 심중로 121']

In [27]:
driver = webdriver.Chrome(path)
driver.get('https://www.airkorea.or.kr/web/pastSearch?pMENU_NO=123')
for i in address_list:
    driver.find_element_by_xpath('//*[@id="s_condition_input3"]').click()
    driver.implicitly_wait(10)

    #chrome switch
    driver.switch_to_window(driver.window_handles[1])
    driver.get_window_position(driver.window_handles[1])
    driver.implicitly_wait(10)

    # site select
    driver.find_element_by_xpath('//*[@id="keyword"]').send_keys(i)
    driver.find_element_by_xpath('//*[@id="serarchContentBox"]/fieldset/span/input[2]').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="roadAddrDiv1"]/b').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="resultData"]/div/a').click()
    time.sleep(3)
    
    #chrome switch
    driver.switch_to_window(driver.window_handles[0])
    driver.get_window_position(driver.window_handles[0])
    driver.find_element_by_xpath('//*[@id="cont_body"]/form/div[1]/a').click()
    time.sleep(3)
    
    # Search(7월)
    driver.find_element_by_xpath('//*[@id="choice_1"]').click()
    driver.find_element_by_xpath('//*[@id="choice_4"]/option[7]').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="cont_body"]/div[1]/a[1]').click()
    time.sleep(2)
    #save file
    driver.find_element_by_xpath('//*[@id="cont_body"]/div[1]/a[3]').click()
    time.sleep(10)
    
    # Search(8월)
    driver.find_element_by_xpath('//*[@id="choice_4"]/option[8]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="cont_body"]/div[1]/a[1]').click()
    time.sleep(2)
    
    #save file
    driver.find_element_by_xpath('//*[@id="cont_body"]/div[1]/a[3]').click()
    time.sleep(10)
    
    
    # Search(9월)

    driver.find_element_by_xpath('//*[@id="choice_4"]/option[9]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="cont_body"]/div[1]/a[1]').click()
    time.sleep(3)
    #save file
    driver.find_element_by_xpath('//*[@id="cont_body"]/div[1]/a[3]').click()
    time.sleep(10)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: use driver.switch_to.window instead


WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.88)


In [25]:
# Selenium Load
driver = webdriver.Chrome(path)
# site connect
driver.get('https://www.airkorea.or.kr/web/realSearch?pMENU_NO=97')

for i in address_list:
    driver.find_element_by_xpath('//*[@id="s_condition_input3"]').click()
    driver.implicitly_wait(10)

    #chrome switch
    driver.switch_to_window(driver.window_handles[1])
    driver.get_window_position(driver.window_handles[1])
    driver.implicitly_wait(10)

    # site select
    driver.find_element_by_xpath('//*[@id="keyword"]').send_keys(i)
    driver.find_element_by_xpath('//*[@id="serarchContentBox"]/fieldset/span/input[2]').click()
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="roadAddrDiv1"]/b').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="resultData"]/div/a').click()
    time.sleep(3)

    #chrome switch
    driver.switch_to_window(driver.window_handles[0])
    driver.get_window_position(driver.window_handles[0])
    driver.find_element_by_xpath('//*[@id="cont_body"]/form[2]/div[1]/a').click()
    time.sleep(3)
    
    #측정자료 조회 날짜 설정(10월1일 ~11월30일)
    driver.find_element_by_xpath('//*[@id="from_date"]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[7]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[10]').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[1]/td[3]/a').click()
    time.sleep(10)
    driver.find_element_by_xpath('//*[@id="to_date"]').click()
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[1]/option[7]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/div[1]/div/select[2]/option[11]').click()
    driver.find_element_by_xpath('//*[@id="ui-datepicker-div"]/table/tbody/tr[5]/td[7]/a').click()
    time.sleep(4)
    driver.find_element_by_xpath('//*[@id="to_date_hour"]/option[24]').click()
    #검색 버튼 클릭
    driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[1]').click()
    time.sleep(5)

    #Save file
    driver.find_element_by_xpath('//*[@id="tab1"]/div[1]/a[3]').click()
    time.sleep(10)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use driver.switch_to.window instead
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: use driver.switch_to.window instead


## 고속도로, 일반도로 자료 크롤링

교통량 정보 제공 시스템 사이트 

통계자료 -> 도로종류별 교통량에 있는 일반국도(52개), 고속국도(48개) 데이터 수집

In [48]:
# Selenium Load
driver = webdriver.Chrome(path)
driver.implicitly_wait(3)
# webdriver start
driver.get('http://www.road.re.kr/itms/itms_01.asp?main_no=11&sub_no=1')
driver.implicitly_wait(10)

# 고속도로 select 48
for i in range(1,49) :
    driver.find_element_by_xpath('//*[@id="search"]/table/tbody/tr/td[4]/label/select').click()
    time.sleep(5)
    driver.find_element_by_xpath(f'//*[@id="search"]/table/tbody/tr/td[4]/label/select/option[{i}]').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="data_table"]/table[2]/tbody/tr/td[2]/a/img').click()
    time.sleep(5)

In [51]:
# Selenium Load
driver = webdriver.Chrome(path)
driver.implicitly_wait(3)
# webdriver start
driver.get('http://www.road.re.kr/itms/itms_01_1.asp?main_no=11&sub_no=2')
driver.implicitly_wait(10)

# 일반도로 select 48
for i in range(1,52) :
    driver.find_element_by_xpath('//*[@id="search"]/table/tbody/tr/td[4]/label/select').click()
    driver.find_element_by_xpath(f'//*[@id="search"]/table/tbody/tr/td[4]/label/select/option[{i}]').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="data_table"]/table[2]/tbody/tr/td[2]/a/img').click()
    time.sleep(3)

In [3]:
# webdriver close
driver.close()

## 중국

In [5]:
driver = webdriver.Chrome(path)

In [3]:
datelist = pd.DataFrame(pd.date_range('2020-01-10',' 2019-12-24'))
datelist.columns = ['date']
datelist['date'] =datelist['date'].astype(str)
datelist['date'] = datelist['date'].str.replace('-','')
datelist = datelist['date'].unique().tolist()

In [9]:
for i in datelist :
    driver.get(f'http://beijingair.sinaapp.com/data/china/sites/{i}/csv')